In [ ]:
# Does not need to be executed if ~/.ipython/profile_default/ipython_config.py
# exists and contains get_config().InteractiveShell.ast_node_interactivity = 'all'

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [ ]:
from math import sqrt
from itertools import chain
from timeit import timeit

Let $n$ be a natural number. If a natural number $m$ at most equal to $n$ is not prime then $m$ is of the form $p_1\times\dots\times p_k$ for some $k\geq 2$ and prime numbers $p_1$, ..., $p_k$ with $p_1\leq\dots\leq p_k$; hence $n\geq m\geq p_1^2$, hence $p_1\leq\sqrt n$. This implies that all natural numbers at most equal to $n$ that are not prime have a proper factor at most equal to $\lfloor\sqrt n\rfloor$. So to identify all prime numbers up to and possibly including $n$, it suffices to cross out, from the collection of all numbers between 2 and $n$, all proper multiples at most equal to $n$ of 2, 3, ... up to and including $\lfloor\sqrt n\rfloor$. Moreover, given a number $p$ at most equal to $\lfloor\sqrt n\rfloor$, if all proper multiples at most equal to $n$ of all numbers greater than 1 and smaller than $p$ have been crossed out, then either $p$ has been crossed out (together with all its multiples at most equal to $n$, case in which $p$ is not prime), or only its proper multiples at least equal to $p^2$ and at most equal to $n$ remain to be crossed out (case in which $p$ is prime).

There is a risk that the computation of $\lfloor\sqrt n\rfloor$ yields a smaller number. The risk seems particularly high in case $n$ is the perfect square of a prime: if the computation of $\lfloor\sqrt n\rfloor$ yielded a smaller number, then $n$ would not be crossed out and be incorrectly part of the collection of integers eventually declared to be prime.

To appreciate the imprecision of floating point computation, let us witness computations of $(\sqrt n)^2$ that are too small, correct (as a floating point number), or too large:

In [ ]:
too_small = []
just_right = []
too_large = []

n = 1
while len(too_small) < 10 or len(just_right) < 10 or len(too_large) < 10:
    sqrt_n = sqrt(n)
    if sqrt_n ** 2 < n and len(too_small) < 10:
        too_small.append((n, sqrt_n, sqrt_n ** 2))
    elif sqrt_n ** 2 == n and len(just_right) < 10:
        just_right.append((n, sqrt_n, sqrt_n ** 2))
    else:
        too_large.append((n, sqrt_n, sqrt_n ** 2))        
    n += 1

print('Too small!')
for triple in too_small:
    print(triple)
print('\nJust right!')
for triple in just_right:
    print(triple)
print('\nToo large!')
for triple in too_large:
    print(triple)

The square roots of the perfect squares that have been considered in the previous code fragment have all been computed correctly (as floating point numbers). Also observe that they have been squared correctly (as floating point numbers), but for large enough perfect squares, that does not hold any more:

In [ ]:
too_small = None
too_large = None

i = 1
while too_small is None or too_large is None:
    i_square = i ** 2
    if sqrt(i_square) ** 2 < i_square:
        too_small = i, i_square, sqrt(i_square), sqrt(i_square) ** 2
    elif sqrt(i_square) ** 2 > i_square:
        too_large = i, i_square, sqrt(i_square), sqrt(i_square) ** 2
    i += 1

print('Too small!')
print(too_small)
print('\nToo large!')
print(too_large)

The previous code fragment leaves open the possibility that the computation of the square root of a perfect square is always correct (as a floating point number), and in particular, is never smaller than $\lfloor\sqrt n\rfloor$. It is also possible that when $n$ is not a perfect square, then the computation of $\sqrt n$, though often incorrect, and in particular often smaller than $\sqrt n$, is still never smaller than $\lfloor\sqrt n\rfloor$. So whether $n$ is a perfect square or not, changing the type of the computation of $\sqrt n$ from floating point to integer might result in a correct computation of $\lfloor\sqrt n\rfloor$. Still, to be on the safe side, it is preferable to use __round()__ rather than __int()__.

 Compare:

In [ ]:
int(3.01), round(3.01)
int(2.99), round(2.99)

A natural question in relation to __round()__ is: for a given integer $k$, what is $k+0.5$ rounded to? It turns out to be the one of $k$ and $k+1$ which is closest to 0:

In [ ]:
round(2.5), round(-2.5)

__round()__ also lets us specify a precision:

In [ ]:
round(1.9876543, 0)
round(1.9876543, 1)
round(1.9876543, 2)
round(1.9876543, 3)
round(1.9876543, 10)

A list __sieve__ of length $n+1$ can be used to record whether $i$ is prime for $2\leq i\leq n$, storing __True__ or __False__ at index $i$ depending on whether $i$ is believed to be prime or not. The first two elements of __sieve__, of index 0 and 1, are unused. To start with, we assume that all numbers are prime.

For illustration purposes, let us fix $n$ to some value, store that value in a variable __n__, and define __sieve__ accordingly.

In [ ]:
n = 37
sieve = [True] * (n + 1)

To print __sieve__'s contents and indexes at various stages of the procedure, we make use of the syntax that formats
decimal numbers imposing a particular field width, if necessary padding with spaces (the default) or with 0's:

In [ ]:
print(f'|{100:2}|{100:3}|{100:4}|{100:5}|{100:6}|')
print(f'|{100:02}|{100:03}|{100:04}|{100:05}|{100:06}|')

x = 100; w = 5
print(f'|{x:{w}}|')
print(f'|{x:0{w}}|')

In [ ]:
def print_sieve_contents_and_indexes():
    for e in sieve:
        print('  T', end = '') if e else print('  F', end = '')
    print()
    for i in range(len(sieve)):
        print(f'{i:3}', end = '')
        
print_sieve_contents_and_indexes()

To cross out all multiples at most equal to $n$ of a prime number $p$, starting with $p^2$ if the multiples at most equal to $n$ of all smaller primes have been crossed out already, we need to generate a sequence of the form $p^2$, $p^2+p$, $p^2+2p$... This is easily done with __range()__:

In [ ]:
# One argument, the ending point, which is excluded.
# The starting point is 0, the default,
# The step is 1, the default
list(range(4))
# Two arguments, the starting point, which is included,
# and the ending point, which is excluded.
# The step is 1, the default
list(range(4, 10))
# Three arguments, the starting point, which is included,
# the ending point, which is excluded, and the step.
list(range(3, 11, 2))
list(range(3, 11, 3))
list(range(11, 3, -2))
list(range(11, 3, -3))

To observe how, with __n__ set to __37__, proper multiples of 2, 3 and 5 are crossed out while 4 and 6 are found out to be crossed out (together with their multiples) already, we successively call the following function with __p__ set to __2__, __3__, __4__, __5__ and __6__, equal to $\lfloor\sqrt 37\rfloor$, as argument.

In [ ]:
def cross_out_proper_multiples(p):
    # We assume that this function will be called in the order
    #   eliminate_proper_multiples(2)
    #   eliminate_proper_multiples(3)
    #   eliminate_proper_multiples(4)
    #   ...
    if not sieve[p]:
        print(f'{p} has been crossed out '
              'as a multiple of a smaller number.'
             )
    else:
        print(f'{p} is not a multiple of a smaller number, '
              'hence it is prime.'
             )
        print('Now crossing out all proper multiples '
              f'of {p} at most equal to {n}.'
             )
        for i in range(p * p, n + 1, p):
            print(f' Crossing out {i}')
            sieve[i] = False
        print_sieve_contents_and_indexes()

In [ ]:
cross_out_proper_multiples(2)

In [ ]:
cross_out_proper_multiples(3)

In [ ]:
cross_out_proper_multiples(4)

In [ ]:
cross_out_proper_multiples(5)

In [ ]:
cross_out_proper_multiples(6)

In [ ]:
print(f'The prime numbers are most equal to {n} are:')
for p in range(2, n + 1):
    if sieve[p]:
        print(f'{p:4}', end = '')

Putting it all together:

In [ ]:
def sieve_of_primes_up_to(n):
    primes_sieve = [True] * (n + 1)
    for p in range(2, round(sqrt(n)) + 1):
        if primes_sieve[p]:
            for i in range(p * p, n + 1, p):
                primes_sieve[i] = False
    return primes_sieve

To display all prime numbers at most equal to $n$, we define two functions. One function, __sequence_and_max_size_from()__, is designed to, from the list returned by __sieve_of_primes_up_to()__, determine and return the corresponding sequence of primes $\sigma$ together with the number of digits $l$ in the last (and largest) prime in the sequence; $\sigma$ and $l$ will be assigned to both arguments, __sequence__ and __max_size__, respectively, of the other function, __nicely_display()__. We will utilise this function again when we implement other sieve methods. It is general enough to nicely display any sequence of data all of which are output with at most __max_size__ many characters. More precisely, __nicely_display()__ has the following features. It outputs at most 80 characters per line. Two spaces precede the display of the data that are output with __max_size__ many characters. Three spaces precede the display of the data that are output with __max_size__ minus 1 many characters, if any. Four spaces precede the display of the data that are output with __max_size__ minus 2 many digits, if any... That way, all data will be nicely aligned column by column, with the guarantee that at least two spaces will separate two consecutive data on the same line. If $l$ is the value of __max_size__, then exactly $\lfloor\frac{80}{l+2}\rfloor$ data will be displayed per line, with the possible exception of the last line:

In [ ]:
def nicely_display(sequence, max_size):
    field_width = max_size + 2
    nb_of_fields = 80 // field_width
    count = 0
    for e in sequence:
        print(f'{e:{field_width}}', end = '')
        count += 1
        if count % nb_of_fields == 0:
            print()
        
nicely_display(range(200), 3)

To determine the value of __max_size__ when using __nicely_display()__ to display all prime numbers up to a largest prime number $p$, we need to determine the number of digits in $p$, which is easily done by letting __str()__ produce a string from an integer, and calling __len()__ on the former:

In [ ]:
str(991)
len(str('991'))

In __nicely_display()__, a __for__ statement processes its first argument, __sequence__. So __sequence__ has to be an iterable, and possibly an iterator. The __next()__ method can be applied to an iterator. From an iterable that is not an iterator, one can get an iterator thanks to the __iter()__ function. The __iter()__ function can be applied to any iterable, so also to an iterator, in which case it just returns its argument:

In [ ]:
# An iterable (an object of the range class)
# that is not an iterator
x = range(2)
x is iter(x)
y = iter(x)
next(y)
next(y)

# An iterable (a list)
# that is not an iterator
x = [10, 11]
x is iter(x)
y = iter(x)
next(y)
next(y)

# An iterable (a generator expression)
# that is an iterator
x = (u for u in (100, 200))
x is iter(x)
next(x)
next(x)

When a __for__ statement processes an iterator, it calls __next__() again and again, until a __StopIteration__ is generated, causing it to gracefully stop execution. When a __for__ statement processes an iterable that is not an iterator, it first gets an iterator from the iterable thanks to __iter()__, iterator which is then processed as described. So the argument __sequence__ of __nicely_display()__ can be either an iterable that is not an iterator, like a list, or a tuple; or it can be an iterator, like a generator expression. The second option can lead to more effective code than the first one. Indeed, when a __for__ statement processes a list or tuple, then that list or tuple had to be created in the first place, which the __for__ statement then processes by implicit calls to __next()__ on an iterator produced from that list or tuple by __iter()__. On the other hand, when a __for__ statement processes a generator expression, then only a mechanism to produce a sequence had to be created in the first place, and that mechanism is activated (__next()__ is implicitly called again and again) to generate all elements in the sequence and process them "on the fly": 

In [ ]:
sieve = [True, True, True, True, False, True, False, True, False]
# A list created from sieve thanks to a list comprehension.
# sieve has been scanned from beginning to end to create primes.
primes = [i for i in range(2, len(sieve)) if sieve[i]]
primes
# An iterator is created from primes, to generate all members of primes
# and print them out.
# So eventually, two sequences will have been processed.
for e in primes:
    print(e, end = ' ')
    
sieve = [True, True, True, True, False, True, False, True, False]
# A generator expression defined from sieve.
# sieve has not been scanned from beginning to end to create primes;
# primes is a mechanism to generate some numbers from sieve.
primes = (i for i in range(2, len(sieve)) if sieve[i])
primes
# The mechanism is activated as the for loop is executed.
# As an effect, sieve is scanned from beginning to end,
# numbers are generated and printed out on the fly.
# So eventually, only one sequence will have been processed.
for e in primes:
    print(e, end = ' ')

Based on these considerations, we define __sequence_and_max_size_from()__ as follows:

In [ ]:
def sequence_and_max_size_from(sieve):
    largest_prime = len(sieve) - 1
    while not sieve[largest_prime]:
        largest_prime -= 1
    return (p for p in range(2, len(sieve)) if sieve[p]),\
           len(str(largest_prime))

We now have everything we need to nicely display all prime numbers at most equal to $n$:

In [ ]:
nicely_display(*sequence_and_max_size_from(sieve_of_primes_up_to(1_000)))

To save half of the sieve's space and not have to cross out the proper multiples of 2, one can change __sieve__ and make it a list of length $\lfloor\frac{n + 1}{2}\rfloor$, with indexes 0, 1, 2, 3, 4, 5... meant to refer to the numbers 2, 3, 5, 7, 9...  The price we pay for this is that we lose the simple equivalence between "number $p$ is prime" and "__sieve__ eventually stores __True__ at index $p$": the equivalence becomes: "number $p$ is prime" iff "__sieve__ eventually stores __True__ at index $\lfloor\frac{p - 1}{2}\rfloor$".

Let $p$ be a number between 3 and $\lfloor\sqrt n\rfloor$. The index that refers to $p$ in this modified __sieve__ is $k=\frac{p-1}{2}$, hence the index that refers to $p^2$ is $\frac{p^2-1}{2}=\frac{(p-1)(p+1)}{2}=\frac{p-1}{2}2(\frac{p-1}{2}+1)=2k(k+1)$. Also, only the proper odd multiples at most equal to $n$ of $p$ have to be crossed out; so after having crossed out such a multiple $a$, the next multiple of $p$ that needs to crossed out (in case it is still at most equal to $n$), is referred to at index $\frac{a+2p-1}{2}=\frac{a-1}{2}+p=\frac{a-1}{2}+2k+1$, so $2k+1$ needs to be added to the index that refers to $a$ to refer to that next multiple of $p$.

Putting it all together:

In [ ]:
def optimised_sieve_of_primes_up_to(n):
    n_index = (n - 1) // 2
    sieve = [True] * (n_index + 1)
    for k in range(1, (round(sqrt(n)) + 1) // 2):
        if sieve[k]:
            for i in range(2 * k * (k + 1), n_index + 1, 2 * k + 1):
                sieve[i] = False
    return sieve

To display all prime numbers at most equal to $n$ from the list returned by __optimised_sieve_of_primes_up_to()__, we need to adapt the function __sequence_and_max_size_from()__. Essentially, one has to generate all numbers of the form $2i+1$ for all $1\leq i\leq\lfloor\frac{n - 1}{2}\rfloor$ such that the list __sieve__ returned by __optimised_sieve_of_primes_up_to()__ has a value of __True__ at index $i$; such is the relationship between the odd prime numbers at most equal to $n$ and the strictly positive indexes in __sieve__. But these odd prime numbers have to be preceded with 2. We still want to return an iterator. The simplest solution is to create an iterator from an iterator meant to generate 2 only, and the generator expression __(2 * p + 1 for p in range(1, len(sieve)) if sieve[p]))__. The __chain()__ function from the __itertools__ module lets us combine a sequence of iterables (some of which can be iterators) into one iterator:

In [ ]:
# Providing as argument to list()
# an iterator created from two iterators
list(chain(iter(range(2)), (i for i in [10, 20, 30])))
# Providing as argument to list()
# an iterator created from one iterator
# and one iterable that is not an iterator
list(chain(range(2), (i for i in [10, 20, 30])))
# Providing as argument to list()
# an iterator created from two iterables
# that are not iterators
list(chain(range(2), [10, 20, 30]))

Based on these considerations, we nicely display all prime numbers identified by  __optimised_sieve_of_primes_up_to()__ as follows:

In [ ]:
def optimised_sequence_and_max_size_from(sieve):
    largest_prime = len(sieve) - 1
    while not sieve[largest_prime]:
        largest_prime -= 1
    return chain((2,),
                 (2 * p + 1 for p in range(1, len(sieve)) if sieve[p])
                ),\
           len(str(largest_prime))

nicely_display(*optimised_sequence_and_max_size_from(
                                optimised_sieve_of_primes_up_to(1_000)
                                                    )
              )

Let us get an idea of how large we can afford $n$ to be and how more efficient __optimised_sieve_of_primes_up_to()__ is compared to __sieve_of_primes_up_to()__. We ask the __timeit()__ method from the __timeit__ module to executing once (__number = 1__) the code __sieve_of_primes_up_to(10_000_000)__, the assignment of __globals()__ to __globals__ being needed to let __timetit()__ know about the names __sieve_of_primes_up_to__ and __optimised_sieve_of_primes_up_to__:

In [ ]:
type(globals())
'sieve_of_primes_up_to' in globals()
'optimised_sieve_of_primes_up_to' in globals()

In [ ]:
timeit('sieve_of_primes_up_to(10_000_000)',
       globals = globals(),
       number = 1
      )
timeit('optimised_sieve_of_primes_up_to(10_000_000)',
       globals = globals(),
       number = 1
      )